Gratidão aos colegas Thiago Soares Laitz e Hugo (hugo@maritaca.ai) pelo apoio e código base fornecidos

# Finetuning Ranker

Gratidão ao código base fornecido pelo colega de curso Thiago Soares Laitz

# Installs

In [ ]:
pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Infra

## Função de verificação de memória

In [ ]:
from psutil import virtual_memory

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jun 23 16:02:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
def mostra_memoria(lista_mem=['cpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'.
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])

  Autor: Marcus Vinícius Borela de Castro

  """
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [ ]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 89.64
 available 87.58
 used 1.12
 free 49.27
 cached 39.09
 buffers 0.17
/nGPU
Fri Jun 23 16:02:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+

## Imports

In [ ]:
import os

In [ ]:
from transformers import Trainer
import numpy as np
import torch


In [ ]:
import pandas as pd

In [ ]:
from datasets import Dataset

In [ ]:
import json
from transformers import (
    AutoConfig,
    TrainingArguments,
    HfArgumentParser,
    MT5ForConditionalGeneration,
    MT5Tokenizer
)
from tqdm.auto import tqdm
from datasets import Dataset
from dataclasses import dataclass, field


## Preparando para debug e display

In [ ]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [ ]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True)
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345,
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite
    uma melhor compreensão dos números envolvidos.
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica,
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada,
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada.
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona,
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  # transformers.utils.logging.set_verbosity_info()


  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a
    necessidade de cada tarefa.

    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning

    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  # %xmode Verbose

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain,
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.

      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha,
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código),
        step (para entrar em uma função chamada na próxima linha de código)
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [ ]:
config_display()

In [ ]:
config_debug()

## Vinculando pasta do google drive para salvar dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /content


# Carga dos dados

read train data

In [ ]:
PATH_LOCAL_DATA = '/content/drive/MyDrive/treinamento/202301_IA368DD/indir/data'

In [ ]:
# path_data = '/content/drive/MyDrive/treinamento/202301_IA368DD/indir/data/train_data_juris_tcu_index_bm25.csv'

PATH_TRAIN_DATA_ZIP = f"{PATH_LOCAL_DATA}/train_data_juris_tcu_index.zip"
PATH_TRAIN_DATA = f"{PATH_LOCAL_DATA}/train_data_juris_tcu_index.csv"

In [ ]:
os.path.exists(PATH_TRAIN_DATA)

True

In [ ]:
!wget https://github.com/marcusborela/ind-ir/raw/main/data/train_juris_tcu_index/train_data_juris_tcu_index.zip -O {PATH_TRAIN_DATA_ZIP}

--2023-06-23 16:02:28--  https://github.com/marcusborela/ind-ir/raw/main/data/train_juris_tcu_index/train_data_juris_tcu_index.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/marcusborela/ind-ir/main/data/train_juris_tcu_index/train_data_juris_tcu_index.zip [following]
--2023-06-23 16:02:28--  https://raw.githubusercontent.com/marcusborela/ind-ir/main/data/train_juris_tcu_index/train_data_juris_tcu_index.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66194499 (63M) [application/zip]
Saving to: ‘/content/drive/MyDrive/treinamento/202301_IA368DD/indir/data/train_data_juris_tcu_index.z

In [ ]:
import zipfile

In [ ]:
%%time
if not os.path.exists(PATH_TRAIN_DATA):
  !wget https://github.com/marcusborela/ind-ir/raw/main/data/train_juris_tcu_index/train_data_juris_tcu_index.csv -O {PATH_TRAIN_DATA}

  # Extrair o arquivo zip
  with zipfile.ZipFile(PATH_TRAIN_DATA_ZIP, 'r') as zip_ref:
      zip_ref.extractall(PATH_LOCAL_DATA)

  # Listar os arquivos extraídos
  arquivos_extraidos = zip_ref.namelist()
  # Exibir os arquivos extraídos
  for arquivo in arquivos_extraidos:
      print(arquivo)
  print("File loaded")
else:
  print("File already there!")

File already there!
CPU times: user 0 ns, sys: 710 µs, total: 710 µs
Wall time: 652 µs


In [ ]:
df = pd.read_csv(PATH_TRAIN_DATA)

In [ ]:
df.shape

(282636, 6)

Verificando correção do arquivo!

In [ ]:
print(df.isnull().sum())

QUERY_ID          0
DOC_ID            0
RELEVANCE         0
TYPE_RELEVANCE    0
DOC_TEXT          0
QUERY_TEXT        0
Length: 6, dtype: int64


In [ ]:
df[['QUERY_TEXT','DOC_TEXT']].applymap(len).describe()

,QUERY_TEXT,DOC_TEXT
count,282636.0000000,282636.0000000
mean,314.4387445,971.9820688
std,159.8934908,671.0375228
min,41.0000000,86.0000000
25%,211.0000000,595.0000000
50%,286.0000000,812.0000000
75%,383.0000000,1162.0000000
max,4212.0000000,3739.0000000


Para cada positivo, tem 2 negativos

In [ ]:
df['RELEVANCE'].describe()

count    282636.0000000
mean          0.3333333
std           0.4714054
min           0.0000000
25%           0.0000000
50%           0.0000000
75%           1.0000000
max           1.0000000
Name: RELEVANCE, Length: 8, dtype: float64

# Treino

## setup

In [ ]:
class T5Trainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        token_false, token_true = ['▁', '▁true']
        self.token_false_id = self.tokenizer.get_vocab()[token_false]
        self.token_true_id  = self.tokenizer.get_vocab()[token_true]

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.
        Subclass and override for custom behavior.
        """
        # labels = np.squeeze(inputs.pop("labels"))
        labels = inputs.pop("labels")
        input_ids = inputs.input_ids
        attention_mask = inputs.attention_mask
        # print(f"em compute_loss input_ids.size(0) {input_ids.size(0)} ")
        decode_ids = torch.full((input_ids.size(0), 1),
                         model.config.decoder_start_token_id,
                         dtype=torch.long).to(input_ids.device)
        encoder_outputs = model.get_encoder()(input_ids, attention_mask=attention_mask)
        # print(f"em compute_loss antes prepare_inputs ")
        model_inputs = model.prepare_inputs_for_generation(
            decode_ids,
            encoder_outputs=encoder_outputs,
            past=None,
            attention_mask=attention_mask,
            # use_cache=True pois é incompatível com checkpoint = True
            use_cache= False
        )
        # print(f"em compute_loss antes chamada model ")
        outputs = model(**model_inputs)
        # print(f"em compute_loss após chamada model ")
        logits = outputs[0][:, :, [self.token_false_id, self.token_true_id]][:, 0]
        # print(f"em compute_loss antes chamada log_softmax ")
        probs = torch.nn.functional.log_softmax(logits, dim=1)
        # print(f"em compute_loss antes chamada cross_entropy \nprobs {probs} \nlabels {labels}")
        loss = torch.nn.functional.cross_entropy(probs, labels) # CONFERIR
        # print(f"saindo do compute_loss")
        return (loss, outputs) if return_outputs else loss


In [ ]:
@dataclass
class ExtraArguments:
    model: str = field(
        metadata={"help": "Pretrained model."},
    )
    training_file: str = field(
        metadata={"help": "JSONL training file. Each line should contain a JSON object with the keys 'query'(str), 'text'(str) and 'label'(0 or 1)."},
    )
    resume_training: bool = field(
        default=False,
        metadata={"help": "Resume training from the last checkpoint and trainer state"}
    )


In [ ]:
!ls

drive  sample_data  trained_model


In [ ]:
MODEL_NAME = 'unicamp-dl/mt5-3B-mmarco-en-pt'

training_args = TrainingArguments(output_dir = "trained_model")
# Needed to make the Trainer work with an on-the-fly transformation on the dataset
training_args.remove_unused_columns = False
training_args.output_dir = "trained_model"
training_args.warmup_steps=1000 # Alterar!
training_args.num_train_epochs=1.0 # Alterar!
training_args.logging_steps=100 # Alterar!
training_args.save_steps=50000 # Alterar!
training_args.learning_rate=7e-5
training_args.per_device_train_batch_size=2 # Alterar!
training_args.gradient_accumulation_steps=2 # Alterar!
training_args._n_gpu = 1
# training_args.bf16 = True # se for usar a100, 3090, 4090 -> usar
training_args.evaluation_strategy = "no"
training_args.do_eval = False
training_args.optim='adamw_bnb_8bit'
training_args.gradient_checkpointing = True
# training_args.report_to="none"

In [ ]:
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=2,
gradient_checkpointing=True,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<

In [ ]:
# from transformers.integrations import NeptuneCallback
# rastro_neptune = NeptuneRastroRun(hparam, parm_lista_tag= tag_contexto_rastro)
# neptune_callback = NeptuneCallback(run=rastro_neptune)

In [ ]:
extra_args = ExtraArguments(
    model="pretrained_model",
    training_file="training_data.jsonl",
    resume_training=False
)

In [ ]:
tokenizer = MT5Tokenizer.from_pretrained(MODEL_NAME)

In [ ]:
config = AutoConfig.from_pretrained(MODEL_NAME)
config.num_labels = 1
config.problem_type = "multi_label_classification"

In [ ]:
%%time
model = MT5ForConditionalGeneration.from_pretrained(
    MODEL_NAME, config=config
)

CPU times: user 41.7 s, sys: 13.8 s, total: 55.5 s
Wall time: 54.4 s


## Data selection

In [ ]:
df.shape

(282636, 6)

In [ ]:
df2 = df # df.head(20000)

In [ ]:
train_examples = {
    'query': df2['QUERY_TEXT'].tolist(),
    'text': df2['DOC_TEXT'].tolist(),
    'label': df2['RELEVANCE'].astype(int).tolist()
}


In [ ]:
train_examples_count = df2.shape[0]

In [ ]:
del df2

In [ ]:
train_examples['query'][0],train_examples['text'][0], train_examples['label'][0]

('SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fundamento na Lei nº 3.738, de 04/04/60, as vantagens previstas no art. 184 da Lei nº 1.711, de 28/10/52.',
 'O termo é "Pessoal".\nPessoal tem nota de escopo: "Designação genérica de todos os servidores ou funcionários civis pertencentes ao quadro de pessoal de um órgão ou entidade.".\nPessoal tem nota de escopo: "Tema agrupador para área de atuação do Controle Externo.".\nPessoal é uma generalização de: "Servidor público", "Funcionário público", "Pessoal civil", "Pessoal militar", "Pessoal temporário" e "Colaborador".\nPessoal tem termo relacionado: "Sisac", "Despesa com pessoal" e "HCAAF".\nPessoal tem tradução em espanhol: "Recursos humanos".\nPessoal tem tradução em inglês: "Personal" e "Human resources".',
 1)

In [ ]:
print(train_examples_count)

282636


In [ ]:
def tokenize(batch):
    queries_documents = [f"Query: {query} Document: {text} Relevant:" for query, text in zip(batch["query"], batch["text"])]
    print(f"Chamado tokenize len(queries_documents): {len(queries_documents)}")
    tokenized = tokenizer(
        queries_documents,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        max_length= 512
    )
    # tokenized["labels"] = [[label] for label in batch["label"]]
    # tokenized['label'] = [[token_false, token_true][int(pairs["label"][i])]
    tokenized["labels"] = [[1., 0.] if label == 0 else [0., 1.] for label in batch["label"]]

    return tokenized


In [ ]:
train_dataset = Dataset.from_dict(train_examples)

In [ ]:
train_dataset.set_transform(tokenize)
"""
outra opção seria carregar em memória
train_dataset = train_dataset.map(
        tokenize,
        remove_columns=('query', 'text', 'label'),
        batched=True,
        desc='Tokenizing',
    )
"""

"\noutra opção seria carregar em memória\ntrain_dataset = train_dataset.map(\n        tokenize,\n        remove_columns=('query', 'text', 'label'),\n        batched=True,\n        desc='Tokenizing',\n    )\n"

In [ ]:
print(train_dataset[0])

Chamado tokenize len(queries_documents): 1
{'input_ids': tensor([   259, 102385,    267,    320,   5928, 193660,    366,  32893,  11230,
           259,  14337,    303,    259, 135246,   2930,    958,  12887, 100415,
           261,    435,    259,    265,  18239,    269, 129488,   1317,    350,
          3617,   3528,  10733,    259,    284,  66124,    268,    294,   8982,
           375,   1501,  37344,    261,    269,   2574,  46268,   2886,    261,
           527,    464,    270,  17048,    421, 147564,    375,   3684,    260,
         48271,    350,   8982,    375,  40218,   1003,    261,    269,   1190,
         30149,   5185,    260,  31681,    267,    443,  29803,    259,    361,
           313,    559,    299,  45018,   1191,  30566,  45018,   2834,  20137,
           269,    655, 136018,    267,    313,  48409,   2134,   3608, 134754,
           269,   1746,    259,    337,  19844,   4172,   1032,  17625,  11023,
         99912,    263,  53463,  81758,    259,    262,    268,

In [ ]:
print('***** Configs *****')
print(f'Problem type:', config.problem_type)
print(f'Training examples:', train_examples_count)
print('*' * 20)


***** Configs *****
Problem type: multi_label_classification
Training examples: 282636
********************


## Train

In [ ]:
raise Exception('Parar aqui reinício!')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
Exception: Parar aqui reinício!

In [ ]:
trainer = T5Trainer(
    model,
    training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

In [ ]:
train_result = trainer.train(resume_from_checkpoint=False)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Chamado tokenize len(queries_documents): 2
Chamado tokenize len(queries_documents): 2


Step,Training Loss


Chamado tokenize len(queries_documents): 2
Chamado tokenize len(queries_documents): 2


/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:200: UserWarning: Error detected in EmbeddingBackward0. Traceback of forward call that caused the error:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  File "/usr/lib/pyth

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1938 in _inner_training_loop     │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2770 in training_step            │
│                                                                                                  │
│   2767 │   │   │   with amp.scale_loss(loss, self.optimizer) as scaled_loss:                     │
│   2768 │   │   │   │   scaled_loss.backward()                                                    │
│   2769 │   │   else:                                                                             │
│ ❱ 2770 │   │   │   self.accelerator.backward(loss)                                               │
│   2771 │   │                                                                                     │
│   2772 │   │   return loss.detach() / self.args.gradient_accumulation_steps                      │
│   2773                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:1821 in backward               │
│                                                                                                  │
│   1818 │   │   elif self.scaler is not None:                                                     │
│   1819 │   │   │   self.scaler.scale(loss).backward(**kwargs)                                    │
│   1820 │   │   else:                                                                             │
│ ❱ 1821 │   │   │   loss.backward(**kwargs)                                                       │
│   1822 │                                                   

In [ ]:
trainer.save_model()
trainer.save_state()